In [1]:
!pip install torchtext==0.6.0
!python -m spacy download de --quiet

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import numpy as np
import spacy
import random
from torchtext.data.metrics import bleu_score
from tqdm import tqdm



class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, drop):
        super(Encoder, self).__init__()

        self.dropout = nn.Dropout(drop)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=drop)

    def forward(self, x):
        embedding = self.embedding(x) # x -> [seq_L, bs]

        embedding_d = self.dropout(embedding) # embedding -> [seq_L, bs, emb_size]
        
        outputs, (hidden, cell) = self.rnn(embedding_d) 
        # outputs -> [seq_L, bs, hidden_size]
        # hidden -> [num_layers, bs, hidden_size]
        # cell -> [num_layers, bs, hidden_size]

        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, drop):
        super(Decoder, self).__init__()

        self.dropout = nn.Dropout(drop)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout= drop)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        x_u = x.unsqueeze(0) # x -> [bs]

        embedding = self.embedding(x_u) # x_u -> [1, bs]

        embedding_d = self.dropout(embedding) # embedding -> [1, bs, emb_size]

        outputs, (hidden, cell) = self.rnn(embedding_d, (hidden, cell))
        # outputs -> [1, bs, hidden_size]
        # hidden -> [2, bs, hidden_size]
        # cell -> [2, bs, hidden_size]

        predictions = self.fc(outputs)
        # predictions -> [1, bs, len(TRG_eng.vocab)]

        predictions = predictions.squeeze(0)
        ## predictions -> [bs, len(TRG_eng.vocab)]

        return predictions, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, source, target, teacher_force_ratio=0.5):
        # source -> [source_len, bs]
        # target -> [target_len, bs]
        batch_size = source.shape[1]
        target_len = target.shape[0]        
        target_vocab_size = len(TRG_eng.vocab)

        hidden, cell = self.encoder(source)

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(self.device)

        x = target[0, :] # grab the first token i.e. <SOS> of all examples of a batch

        for t in range(1, target_len):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, cell = self.decoder(x, hidden, cell)

            # Store next output prediction
            outputs[t] = output

            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(1)

            x = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs

def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for _ , batch in tqdm(enumerate(iterator), total=len(iterator)):
        inp_data = batch.src.to(device)           
        target = batch.trg.to(device)

        output = model(inp_data, target) # output -> [target/out_len, bs, len(english vocab)]

        output = output[1:].reshape(-1, output.shape[2]) 
        # output[1:] -> ignore [0] the token <SOS>
        # output size after reshape -> [target/op len * bs , len(english vocab)]

        target = target[1:].reshape(-1) # target after reshape -> [target/op len * bs]

        optimizer.zero_grad()
        loss = criterion(output, target)
        loss.backward()

        # Clip to avoid exploding gradient issues
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm= clip)
        
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():

        for _,batch in tqdm(enumerate(iterator), total=len(iterator)):
            inp_data = batch.src.to(device)           
            target = batch.trg.to(device)

            output = model(inp_data, target, teacher_force_ratio= 0)  # turn off teacher force

            output = output[1:].reshape(-1, output.shape[2])
            target = target[1:].reshape(-1)

            loss = criterion(output, target)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def translate_sentence(model, sentence, SRC , TRG, device, max_length=50):
    spacy_ger = spacy.load("de")

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
        
    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, SRC.init_token)
    tokens.append(SRC.eos_token)

    # Go through each german token and convert to an index
    text_to_indices = [SRC.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)


    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.encoder(sentence_tensor)

    outputs = [TRG.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == TRG.vocab.stoi["<eos>"]:
            break

    translated_sentence = [TRG.vocab.itos[idx] for idx in outputs]

    # remove start token
    return translated_sentence[1:]


def bleu(data, model, SRC, TRG, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, SRC, TRG, device)
        prediction = prediction[:-1]  # remove <eos> token
        outputs.append(prediction)

        targets.append([trg])
        
    return bleu_score(outputs, targets)


if __name__ == "__main__" :

    SRC_ger = Field(tokenize='spacy', tokenizer_language='de', lower=True, init_token="<sos>", eos_token="<eos>")
    TRG_eng = Field(tokenize='spacy', tokenizer_language='en', lower=True, init_token="<sos>", eos_token="<eos>")

    train_data, valid_data, test_data = Multi30k.splits(exts=(".de", ".en"), fields=(SRC_ger, TRG_eng))

    SRC_ger.build_vocab(train_data, max_size=10000, min_freq=2)
    TRG_eng.build_vocab(train_data, max_size=10000, min_freq=2)

    # print(SRC_ger.vocab.stoi), print(TRG_eng.vocab.stoi)

    num_epochs = 100
    learning_rate = 0.001
    batch_size = 64
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('device:',device)
    input_size_encoder = len(SRC_ger.vocab)
    input_size_decoder = len(TRG_eng.vocab)
    output_size = len(TRG_eng.vocab)
    encoder_embedding_size = 300
    decoder_embedding_size = 300
    hidden_size = 1024  # Needs to be the same for both encoder and decoder RNNs
    num_layers = 2
    enc_dropout = 0.5
    dec_dropout = 0.5
    sort_key=lambda x: len(x.src)
    pad_idx = TRG_eng.vocab.stoi["<pad>"]
    sentence = "ein boot mit mehreren männern darauf wird von einem großen pferdegespann ans ufer gezogen."


    train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
        (train_data, valid_data, test_data),
        batch_size=batch_size,
        sort_within_batch=True,
        sort_key=sort_key,
        device=device,
    )

    encoder_net = Encoder(input_size_encoder, encoder_embedding_size, hidden_size, num_layers, enc_dropout).to(device)

    decoder_net = Decoder(input_size_decoder, decoder_embedding_size, hidden_size, output_size, num_layers, dec_dropout).to(device)

    model = Seq2Seq(encoder_net, decoder_net, device).to(device)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)


    for epoch in range(num_epochs):
        print('---------------------------------------------------------')
        print(f"[Epoch {epoch + 1} / {num_epochs}]")
        
        print('Input German sentence:', sentence)
        translated_sentence = translate_sentence(model, sentence, SRC_ger , TRG_eng, device, max_length=50)
        print('\nTranslated Sentence:',' '.join(translated_sentence))
        
        train_loss = train(model, train_iterator, optimizer, criterion, clip=0.5)
        val_loss = evaluate(model, valid_iterator, criterion)
                
        print(f'Epoch train loss: {train_loss}, validation loss: {val_loss}') 
        
        if (epoch+1)%15 == 0:
            save_path = f'/content/drive/My Drive/Jovian/mt/epoch{epoch+1}.pth'
            torch.save(model, save_path)

    save_path = f'/content/drive/My Drive/Jovian/mt/final.pth'
    torch.save(model, save_path)
    score = bleu(test_data[1:100], model, SRC_ger, TRG_eng, device)
    print(f"Bleu score: {score*100}")

device: cuda
---------------------------------------------------------
[Epoch 1 / 100]
Input German sentence: ein boot mit mehreren männern darauf wird von einem großen pferdegespann ans ufer gezogen.


  0%|          | 0/454 [00:00<?, ?it/s]


Translated Sentence: chased motorbikes worked leg bedroom language enthusiast haircut motorbikes motorbikes ankle train leg overlook bringing pass pass deer pouting taught hammers husband hawaiian runway runway op squash squash haircut haircut bringing cutouts bandanna train electrical bandanna chased bracelet travel bracelet pass east closer sodas sodas range ultimate promoting starting starting


100%|██████████| 16/16 [00:00<00:00, 16.08it/s]


Epoch train loss: 4.552114026662012, validation loss: 4.427948445081711
---------------------------------------------------------
[Epoch 2 / 100]
Input German sentence: ein boot mit mehreren männern darauf wird von einem großen pferdegespann ans ufer gezogen.


...

Epoch train loss: 0.24868689214047618, validation loss: 7.000535815954208
---------------------------------------------------------
[Epoch 100 / 100]
Input German sentence: ein boot mit mehreren männern darauf wird von einem großen pferdegespann ans ufer gezogen.


  0%|          | 0/454 [00:00<?, ?it/s]


Translated Sentence: a boat carrying several men is pulled to shore by a large team of horses . <eos>


100%|██████████| 16/16 [00:00<00:00, 16.01it/s]


Epoch train loss: 0.250396932266029, validation loss: 6.987092316150665
Bleu score: 19.521733011198553


In [ ]:
import jovian
jovian.commit(project='machine-translation')

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
